## 데이터 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/ColabNotebooks/sesac_딥러닝/project

/content/gdrive/MyDrive/ColabNotebooks/sesac_딥러닝/project


In [3]:
import pandas as pd

# pd.set_option('display.max_rows', None) 

comment = pd.read_csv('./labeled_data.csv')

## 데이터 전처리
- 영어제거
- 특수문자 제거
- 한글인지 확인

In [4]:
# 영어 및 특수 문자 제거
import re 

def cleanText(original):
    text = re.sub('[-=+,#/\?:;^-❛˓◞\n\r˂̵✧$.@*\"⭐�🚩💟※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', str(original))
    text = re.sub('[a-zA-z]','', text)
    return text

comment['data'] = comment['data'].apply(cleanText)
comment

,data,label
0,힛갤가고서 힛갤곤볼한번 받아보고싶었는데 하루늦어서이걸못받네좀빨리만들껄,0.0
1,3줄요약 ㄷㄷ,0.0
2,ㅋㅋ,0.0
3,힛갤뱃지 나도줘요,0.0
4,나도줘,0.0
...,...,...
53749,진짜 왜 공중화장실들은 소변기가 바로 직빵으로 보이게 되어있는거냐 문만 살짝살짝 열...,NaN
53750,너무멋있어,NaN
53751,솔직히 남자화장실 다 보인느거 ㅈ가틈,NaN
53752,6985추,NaN


In [5]:
# 라벨링 하지 않은 줄 제거
comment = comment.dropna(axis=0)

# 공백확인
comment['data'].isnull().sum() 

0

In [6]:
# 한글인지 확인
import sys

def is_hangul(text):
    hanCount = len(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', text))
    return hanCount > 0

comment['is_hangul'] = comment['data'].apply(is_hangul)
not_hangul = comment[comment['is_hangul'] == False].index
comment = comment.drop(not_hangul)
comment.reset_index(drop=True, inplace=True)
comment

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,data,label,is_hangul
0,힛갤가고서 힛갤곤볼한번 받아보고싶었는데 하루늦어서이걸못받네좀빨리만들껄,0.0,True
1,3줄요약 ㄷㄷ,0.0,True
2,ㅋㅋ,0.0,True
3,힛갤뱃지 나도줘요,0.0,True
4,나도줘,0.0,True
...,...,...,...
4666,뭔 퀘하고왔다고 자리라한 새끼 기억나네 인기도 내리고 튐ㅋㅋㅋ,1.0,True
4667,응우옌고로시 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,0.0,True
4668,이게 뭔데,0.0,True
4669,몇신지 보기도 힘들겠다,0.0,True


In [7]:
# 욕 문장과 일반 문장 개수 확인
comment['label'].value_counts()

0.0    3812
1.0     859
Name: label, dtype: int64

## 임베딩

In [8]:
# 자모 단위로 분리 
CHOSUNGS = [u'ㄱ',u'ㄲ',u'ㄴ',u'ㄷ',u'ㄸ',u'ㄹ',u'ㅁ',u'ㅂ',u'ㅃ',u'ㅅ',u'ㅆ',u'ㅇ',u'ㅈ',u'ㅉ',u'ㅊ',u'ㅋ',u'ㅌ',u'ㅍ',u'ㅎ']
JOONGSUNGS = [u'ㅏ',u'ㅐ',u'ㅑ',u'ㅒ',u'ㅓ',u'ㅔ',u'ㅕ',u'ㅖ',u'ㅗ',u'ㅘ',u'ㅙ',u'ㅚ',u'ㅛ',u'ㅜ',u'ㅝ',u'ㅞ',u'ㅟ',u'ㅠ',u'ㅡ',u'ㅢ',u'ㅣ']
JONGSUNGS = [u'',u'ㄱ',u'ㄲ',u'ㄳ',u'ㄴ',u'ㄵ',u'ㄶ',u'ㄷ',u'ㄹ',u'ㄺ',u'ㄻ',u'ㄼ',u'ㄽ',u'ㄾ',u'ㄿ',u'ㅀ',u'ㅁ',u'ㅂ',u'ㅄ',u'ㅅ',u'ㅆ',u'ㅇ',u'ㅈ',u'ㅊ',u'ㅋ',u'ㅌ',u'ㅍ',u'ㅎ']

NUM_CHOSUNGS = 19
NUM_JOONGSUNGS = 21
NUM_JONGSUNGS = 28

FIRST_HANGUL_UNICODE = 0xAC00 #'가'
LAST_HANGUL_UNICODE = 0xD7A3 #'힣'

def decompose(hangul_letter): 

    if ord('ㄱ') <= ord(hangul_letter) <= ord('ㆆ'):
        return [hangul_letter]
    elif ord('ㅏ') <= ord(hangul_letter) <= ord('ᆢ'):
        return [hangul_letter]
    if len(hangul_letter) < 1:
        raise NotLetterException('')
    elif not is_hangul(hangul_letter):
        raise NotHangulException('')
    
    code = ord(hangul_letter) - FIRST_HANGUL_UNICODE
    jongsung_index = code % NUM_JONGSUNGS
    code /= NUM_JONGSUNGS
    code = int(code)
    joongsung_index = code % NUM_JOONGSUNGS
    code /= NUM_JOONGSUNGS
    code = int(code)
    chosung_index = code
    
    return (CHOSUNGS[chosung_index], JOONGSUNGS[joongsung_index], JONGSUNGS[jongsung_index])

class NotHangulException(Exception):
    pass

class NotLetterException(Exception):
    pass

class NotWordException(Exception):
    pass


# embedding
# 문장 입력받아서 분리
def decompose_string(text): 
    result = []
    for c in text:
        if is_hangul(c):
            result += [decompose(c)]
    return result

In [9]:
comment['decompose_string'] = comment['data'].apply(decompose_string)
comment['decompose_string'] 

0       [(ㅎ, ㅣ, ㅅ), (ㄱ, ㅐ, ㄹ), (ㄱ, ㅏ, ), (ㄱ, ㅗ, ), (ㅅ,...
1              [(ㅈ, ㅜ, ㄹ), (ㅇ, ㅛ, ), (ㅇ, ㅑ, ㄱ), [ㄷ], [ㄷ]]
2                                              [[ㅋ], [ㅋ]]
3       [(ㅎ, ㅣ, ㅅ), (ㄱ, ㅐ, ㄹ), (ㅂ, ㅐ, ㅅ), (ㅈ, ㅣ, ), (ㄴ...
4                          [(ㄴ, ㅏ, ), (ㄷ, ㅗ, ), (ㅈ, ㅝ, )]
                              ...                        
4666    [(ㅁ, ㅝ, ㄴ), (ㅋ, ㅞ, ), (ㅎ, ㅏ, ), (ㄱ, ㅗ, ), (ㅇ, ...
4667    [(ㅇ, ㅡ, ㅇ), (ㅇ, ㅜ, ), (ㅇ, ㅖ, ㄴ), (ㄱ, ㅗ, ), (ㄹ,...
4668            [(ㅇ, ㅣ, ), (ㄱ, ㅔ, ), (ㅁ, ㅝ, ㄴ), (ㄷ, ㅔ, )]
4669    [(ㅁ, ㅕ, ㅊ), (ㅅ, ㅣ, ㄴ), (ㅈ, ㅣ, ), (ㅂ, ㅗ, ), (ㄱ,...
4670    [(ㅆ, ㅣ, ㅂ), (ㄷ, ㅓ, ㄱ), (ㅂ, ㅗ, ), (ㄱ, ㅗ, ), (ㅂ,...
Name: decompose_string, Length: 4671, dtype: object

In [ ]:
jamo = comment['decompose_string']
comment['decompose_string'][0]

In [11]:
def to_index_array(decomposed, dict):
    result = []
    for c in decomposed:
        for jamo in c:
            result.append(dict[jamo])
    return result

In [47]:
import csv
import torch
import torch.nn as nn

jamo_dictionary = dict()

def add_to_dict(decomposed):
    for c in decomposed:
        for jamo in c:
            if jamo not in jamo_dictionary:
                jamo_dictionary[jamo] = 1 + len(jamo_dictionary)

def padding(arr, N):
    l = len(arr)
    if l > N:
        return arr[:N]
    else:
        for i in range(N-l):
            arr.append(0)
        return arr

def to_embedded_tensor():
    strings = []
    labels = []
    for idx, line in comment.iterrows():
        s = line[0]
        decomposed = decompose_string(s)
        strings.append(decomposed)
        labels.append(int(line[1]))
        add_to_dict(decomposed)
    maxlen = 0
    for i in range(len(strings)):
        strings[i] = to_index_array(strings[i], jamo_dictionary)
        l = len(strings[i])
        maxlen = l if l > maxlen else maxlen

    # padding
    for i in range(len(strings)):
        strings[i] = padding(strings[i], maxlen)

    return strings, labels, (len(jamo_dictionary)+1)

## 모델 학습

In [14]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt

In [48]:
N_TESTDATA = 100
N_EPOCH = 24

x, y, words = to_embedded_tensor()
x, y = torch.LongTensor(x), torch.FloatTensor(y)
x_train = x[N_TESTDATA:]
y_train = y[N_TESTDATA:]
x_test = x[:N_TESTDATA]
y_test = y[:N_TESTDATA]

# print(x_train, y_train)
# print(x_test, y_test)
print(words)

51


In [49]:
# 워드 임베딩
import torch.nn as nn

# torch.nn.Embedding(num_embeddings, embedding_dim, padding_idx=None, max_norm=None, norm_type=2.0, scale_grad_by_freq=False, sparse=False, _weight=None)
embedding_layer = nn.Embedding(num_embeddings = words, embedding_dim = 300, padding_idx = 1)
embedding_layer

Embedding(51, 300, padding_idx=1)

In [50]:
input_size = words
output_size = words
num_epochs = 2000
print_every = 100
plot_every = 10

# chunk에 대한 설명은 아래 함수정의하면서 하겠습니다.
chunk_len = 200

hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

In [51]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.rnn = nn.RNN(self.embedding_size,self.hidden_size,self.num_layers)
        self.decoder = nn.Linear(self.hidden_size, self.output_size)
        
    
    def forward(self, input):
        out = self.encoder(input.view(1,-1))
        # out,hidden = self.rnn(out,hidden)
        out = self.decoder(out.view(batch_size,-1))
        return out,hidden

    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden
    
# model = RNN(input_size, embedding_size, hidden_size, output_size, num_layers)

In [52]:
model = RNN(input_size=words, 
            embedding_size=embedding_size,
            hidden_size=hidden_size, 
            output_size=words, 
            num_layers=2)

In [53]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

In [54]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [44]:
# 모델 생성
model = Net(words, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [ ]:
output = model(x_train)

In [ ]:
acc_list = []

for i in range(num_epochs):
    # 랜덤한 텍스트 덩어리를 샘플링하고 이를 인덱스 텐서로 변환합니다. 
    inp,label = random_training_set()
    hidden = model.init_hidden()

    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y,hidden = model(x,hidden)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        acc_list.append(loss/chunk_len)
        test()
        print("\n","="*100)

## 모델 테스트

In [ ]:
import pickle
import tensorflow as tf
import numpy as np
# from keras.preprocessing import sequence

MAX_LEN = 681
with open('jamo.pydict', 'rb') as f:
    jamodict = pickle.load(f)

def encode_review(text):
    text = decompose_string(text)
    text = to_index_array(text, jamodict)
    text = padding(text, MAX_LEN)
    return text

def predict_test(text):
    model = tf.keras.models.load_model('./latest-yok-detect-model.h5')
    graph =  tf.compat.v1.get_default_graph() #  tf.compat.v1.get_default_graph

    indices = encode_review(text)
    indices = np.array([indices])
    with graph.as_default():
        # result = model.predict_classes(indices)
        result = model.predict(indices) 
        result = result.argmax(axis=-1)

    yok_type = ''
    if result == 0:
        yok_type = '욕아님'
    else:
        yok_type = '욕'
    return yok_type

In [ ]:
yok_type = predict_test('이게뭔데') # 테스트 데이터 따로 만들기
yok_type

In [ ]:
! pip install --upgrade tensorflow==2.5.0